In [3]:
import nba_api.stats.endpoints as nbaapi
import rdshelp
import pandas as pd

# ✅ RDS CONNECTION NOW WORKING! (Fixed security group)
print("🔍 Testing RDS connection after security group fix...")

conn = rdshelp.connect_to_rds('thebigone', 'ajwin', 'CharlesBark!23', 'nba-rds-instance.c9wwc0ukkiu5.us-east-1.rds.amazonaws.com')


🔍 Testing RDS connection after security group fix...
Connected to RDS PostgreSQL database


In [ ]:
gamefinder = nbaapi.leaguegamefinder.LeagueGameFinder(
                            league_id_nullable='00',
                            season_nullable='2024-2025')

In [16]:
gamefinder.get_data_frames()[0].to_csv('test.csv')

In [ ]:
# Query all public tables in the connected RDS database

query = """
SELECT table_name
FROM information_schema.tables
WHERE table_schema = 'public'
ORDER BY table_name;
"""

with conn.cursor() as cur:
    cur.execute(query)
    tables = cur.fetchall()
    table_list = [row[0] for row in tables]
    print("Public tables in the database:")
    print(table_list)


Public tables in the database:
['boxscoreadvancedv3_playerstats', 'gleague_games', 'gleague_players', 'gleague_teams', 'mastergames', 'masterplayers', 'nba_games', 'nba_players', 'nba_teams', 'test_games_g_league', 'test_games_nba', 'test_games_wnba', 'wnba_games', 'wnba_players', 'wnba_teams']


In [4]:
# Query all games from test_games_nba and display as a DataFrame
query = "SELECT * FROM nba_games;"

games_df = None
with conn.cursor() as cur:
    cur.execute(query)
    rows = cur.fetchall()
    colnames = [desc[0] for desc in cur.description]
    games_df = pd.DataFrame(rows, columns=colnames)
    print(f"Pulled {len(games_df)} rows from test_games_nba.")
    display(games_df.head())


Pulled 52782 rows from test_games_nba.


,game_id,season_id,game_date,home_team_id,home_team_abbreviation,home_team_name,away_team_id,away_team_abbreviation,away_team_name,matchup,home_wl,away_wl,season_type,league_name,created_at,updated_at
0,0022301197,002023-24,2024-04-14,1610612759,SAS,San Antonio Spurs,1610612765,DET,Detroit Pistons,DET @ SAS,W,L,Regular Season,NBA,2025-08-11 18:52:21.891137,2025-08-11 18:52:21.891137
1,0022301199,002023-24,2024-04-14,1610612745,HOU,Houston Rockets,1610612746,LAC,LA Clippers,LAC vs. HOU,W,L,Regular Season,NBA,2025-08-11 18:52:21.891137,2025-08-11 18:52:21.891137
2,0022301187,002023-24,2024-04-14,1610612739,CLE,Cleveland Cavaliers,1610612766,CHA,Charlotte Hornets,CHA @ CLE,L,W,Regular Season,NBA,2025-08-11 18:52:21.891137,2025-08-11 18:52:21.891137
3,0022301186,002023-24,2024-04-14,1610612764,WAS,Washington Wizards,1610612738,BOS,Boston Celtics,BOS vs. WAS,L,W,Regular Season,NBA,2025-08-11 18:52:21.891137,2025-08-11 18:52:21.891137
4,0022301190,002023-24,2024-04-14,1610612741,CHI,Chicago Bulls,1610612752,NYK,New York Knicks,NYK vs. CHI,L,W,Regular Season,NBA,2025-08-11 18:52:21.891137,2025-08-11 18:52:21.891137


In [7]:
games_df['game_date'] = pd.to_datetime(games_df['game_date'])

In [8]:
# Check data types for all columns in all tables
query = """
SELECT 
    table_name,
    column_name,
    data_type,
    character_maximum_length,
    is_nullable,
    column_default
FROM information_schema.columns 
WHERE table_schema = 'public'
ORDER BY table_name, ordinal_position;
"""

with conn.cursor() as cur:
    cur.execute(query)
    columns = cur.fetchall()
    colnames = [desc[0] for desc in cur.description]
    schema_df = pd.DataFrame(columns, columns=colnames)
    
print(f"Found {len(schema_df)} columns across all tables")
display(schema_df)

Found 138 columns across all tables


,table_name,column_name,data_type,character_maximum_length,is_nullable,column_default
0,gleague_games,game_id,character varying,20.0,NO,None
1,gleague_games,season_id,character varying,20.0,NO,None
2,gleague_games,game_date,date,NaN,NO,None
3,gleague_games,home_team_id,character varying,20.0,NO,None
4,gleague_games,home_team_abbreviation,character varying,10.0,YES,None
...,...,...,...,...,...,...
133,wnba_teams,conference,character varying,20.0,YES,None
134,wnba_teams,division,character varying,50.0,YES,None
135,wnba_teams,is_active,boolean,NaN,YES,true
136,wnba_teams,created_at,timestamp without time zone,NaN,YES,CURRENT_TIMESTAMP


In [9]:
# Check data types for a specific table (e.g., nba_games)
table_name = 'nba_games'  # Change this to any table you want to inspect

query = f"""
SELECT 
    column_name,
    data_type,
    character_maximum_length,
    numeric_precision,
    numeric_scale,
    is_nullable,
    column_default
FROM information_schema.columns 
WHERE table_schema = 'public' 
AND table_name = '{table_name}'
ORDER BY ordinal_position;
"""

with conn.cursor() as cur:
    cur.execute(query)
    columns = cur.fetchall()
    colnames = [desc[0] for desc in cur.description]
    table_schema_df = pd.DataFrame(columns, columns=colnames)
    
print(f"Schema for table '{table_name}':")
display(table_schema_df)

Schema for table 'nba_games':


,column_name,data_type,character_maximum_length,numeric_precision,numeric_scale,is_nullable,column_default
0,game_id,character varying,20.0,None,None,NO,None
1,season_id,character varying,20.0,None,None,NO,None
2,game_date,date,NaN,None,None,NO,None
3,home_team_id,character varying,20.0,None,None,NO,None
4,home_team_abbreviation,character varying,10.0,None,None,YES,None
5,home_team_name,character varying,100.0,None,None,YES,None
6,away_team_id,character varying,20.0,None,None,NO,None
7,away_team_abbreviation,character varying,10.0,None,None,YES,None
8,away_team_name,character varying,100.0,None,None,YES,None
9,matchup,character varying,50.0,None,None,YES,None


In [ ]:
# More detailed schema information including constraints and indexes
query = """
SELECT 
    t.table_name,
    c.column_name,
    c.data_type,
    CASE 
        WHEN c.character_maximum_length IS NOT NULL THEN c.data_type || '(' || c.character_maximum_length || ')'
        WHEN c.numeric_precision IS NOT NULL AND c.numeric_scale IS NOT NULL THEN c.data_type || '(' || c.numeric_precision || ',' || c.numeric_scale || ')'
        WHEN c.numeric_precision IS NOT NULL THEN c.data_type || '(' || c.numeric_precision || ')'
        ELSE c.data_type
    END as full_data_type,
    c.is_nullable,
    c.column_default,
    CASE WHEN pk.column_name IS NOT NULL THEN 'PRIMARY KEY' ELSE '' END as key_type
FROM information_schema.tables t
JOIN information_schema.columns c ON t.table_name = c.table_name
LEFT JOIN (
    SELECT ku.table_name, ku.column_name
    FROM information_schema.table_constraints tc
    JOIN information_schema.key_column_usage ku 
        ON tc.constraint_name = ku.constraint_name
    WHERE tc.constraint_type = 'PRIMARY KEY'
    AND tc.table_schema = 'public'
) pk ON c.table_name = pk.table_name AND c.column_name = pk.column_name
WHERE t.table_schema = 'public'
ORDER BY t.table_name, c.ordinal_position;
"""

with conn.cursor() as cur:
    cur.execute(query)
    detailed_columns = cur.fetchall()
    colnames = [desc[0] for desc in cur.description]
    detailed_schema_df = pd.DataFrame(detailed_columns, columns=colnames)
    
print("Detailed schema information for all tables:")
print("=" * 50)

# Group by table for better readability
for table in detailed_schema_df['table_name'].unique():
    table_data = detailed_schema_df[detailed_schema_df['table_name'] == table]
    print(f"\n📋 TABLE: {table}")
    print("-" * 30)
    display(table_data[['column_name', 'full_data_type', 'is_nullable', 'key_type']].head(10))